In [1]:
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error 

from kan import *
import warnings
import sys
sys.path.append('../utils')
from treasury_base import *

warnings.filterwarnings("ignore")

torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [16]:
def direct_pred_retrieval():
    data = treasury_data_retrieval('us_treasury_rates_large.csv')
    data = data.set_index('Date')
    targets = data.columns
    window_list = [1, 3, 5]
    lag_list = [1]
    shift_list = [_ for _ in range(20)]

    for shift in shift_list:
        for col in targets:
            data[f'{col}_+_{shift}'] = data[col].shift(-shift)

    for lag in lag_list:
        for window in window_list:
            for col in targets:
                data[f'{col}_-_{lag}_window_{window}'] = data[col].shift(1).rolling(window).mean()
    return data, targets

def train_mse():
    predictions = model(dataset['train_input'])  # Model predictions
    mse = F.mse_loss(predictions, dataset['train_label'], reduction='mean')  # Compute MSE
    return mse ** 0.5  # Return scalar MSE value

def test_mse():
    predictions = model(dataset['test_input']) # Model predictions
    mse = F.mse_loss(predictions, dataset['test_label'], reduction='mean')  # Compute MSE
    return mse ** 0.5

In [17]:
data, ori_col = direct_pred_retrieval()
all_cols = data.columns
data.tail()

,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,...,3 Mo_-_1_window_5,6 Mo_-_1_window_5,1 Yr_-_1_window_5,2 Yr_-_1_window_5,3 Yr_-_1_window_5,5 Yr_-_1_window_5,7 Yr_-_1_window_5,10 Yr_-_1_window_5,20 Yr_-_1_window_5,30 Yr_-_1_window_5
Date,,,,,,,,,,,,,,,,,,,,,
2024-12-02,4.75,4.63,4.51,4.43,4.30,4.17,4.11,4.08,4.13,4.19,...,4.608,4.444,4.360,4.222,4.202,4.160,4.214,4.282,4.546,4.466
2024-12-03,4.66,4.56,4.49,4.40,4.27,4.17,4.13,4.11,4.17,4.23,...,4.584,4.438,4.336,4.182,4.160,4.116,4.170,4.238,4.504,4.418
2024-12-04,4.65,4.54,4.47,4.38,4.23,4.13,4.09,4.07,4.13,4.19,...,4.558,4.426,4.316,4.174,4.144,4.104,4.162,4.230,4.498,4.408
2024-12-05,4.59,4.53,4.46,4.38,4.23,4.15,4.10,4.07,4.12,4.17,...,4.530,4.412,4.288,4.158,4.120,4.084,4.140,4.208,4.476,4.382
2024-12-06,4.57,4.50,4.42,4.34,4.19,4.10,4.05,4.03,4.09,4.15,...,4.502,4.402,4.266,4.150,4.106,4.076,4.130,4.192,4.458,4.360


In [21]:
test_size = 5
df_train, df_test = data[-test_size-500:-test_size], data[-test_size:]
len_train = len(df_train)
pred = list()

for h in range(0, 5):
    if h == 0:
        target_col = ori_col
    else:
        target_col = [f'{element}_+_{h}' for element in ori_col]
    
    feature_col = [element for element in all_cols if 'window' in element]

    df_train_modified = df_train[:(len_train-h)]
    df_test_modified = df_test.iloc[[0]]
    print(len(df_train_modified))

    X_train, y_train = df_train_modified[feature_col], df_train_modified[target_col]
    X_test, y_test = df_test_modified[feature_col], df_test.iloc[h][ori_col]

    n_inputs = X_train.shape[1]
    n_outputs = y_train.shape[1]

    dataset = dict()
    dtype = torch.get_default_dtype()
    dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype).to(device)
    dataset['train_label'] = torch.from_numpy(y_train.values).type(dtype).to(device)
    dataset['test_input'] = torch.from_numpy(X_test.values).type(dtype).to(device)
    dataset['test_label'] = torch.from_numpy(y_test.values).type(dtype).to(device)

    # Initialize the model
    model = KAN(width=[n_inputs, 32, n_outputs], grid=3, k=2, seed=42, device=device)

    # Train the model and compute metrics
    results = model.fit(dataset, opt="Adam", lamb=0.0001, lr=0.001, steps=500, metrics=(train_mse, test_mse))

    # loss_fn = loss_fn_eval = lambda x, y: torch.mean((x - y) ** 2)
    # p = 
    # train_loss = loss_fn(p, dataset['train_label'])
    # print(train_loss)
    # torch.sqrt(train_loss).cpu().detach().numpy()


    pred.append(model.forward(dataset['test_input']).cpu().detach().numpy().flatten())
    # print(n_inputs, n_outputs)

# df_train_modified
# df_test_modified
# y_train
# X_test
# model(dataset['test_input'])

500
checkpoint directory created: ./model
saving model version 0.0


description:   0%|                                                          | 0/500 [00:00<?, ?it/s]

| train_loss: 8.29e-02 | test_loss: 4.34e-01 | reg: 1.20e+02 | : 100%|█| 500/500 [00:36<00:00, 13.66


saving model version 0.1
499
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 9.95e-02 | test_loss: 3.58e-01 | reg: 1.20e+02 | : 100%|█| 500/500 [00:36<00:00, 13.83


...

saving model version 0.1
497
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 1.22e-01 | test_loss: 3.04e-01 | reg: 1.20e+02 | : 100%|█| 500/500 [00:36<00:00, 13.83


saving model version 0.1
496
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 1.31e-01 | test_loss: 2.89e-01 | reg: 1.20e+02 | : 100%|█| 500/500 [00:36<00:00, 13.58


saving model version 0.1


In [22]:
real = df_test[ori_col].values
mean_squared_error(real, pred, squared=False)

np.float64(0.0709152898217905)

In [23]:
naive = [df_train[ori_col].iloc[-1].values] * test_size
mean_squared_error(real, naive, squared=False)

np.float64(0.05929291173832928)

In [24]:
pred

[array([4.6667267 , 4.68927918, 4.5853632 , 4.504331  , 4.27814938,
        4.20824851, 4.09112449, 4.081449  , 4.163842  , 4.25314148,
        4.51154683, 4.40864724]),
 array([4.66099656, 4.68592407, 4.58172234, 4.50490309, 4.27555263,
        4.21414699, 4.09317448, 4.07995467, 4.16183201, 4.25283188,
        4.50764412, 4.40571988]),
 array([4.66014357, 4.68471283, 4.5817211 , 4.50881273, 4.2757498 ,
        4.22193954, 4.09657252, 4.08054528, 4.16124849, 4.25463506,
        4.50647177, 4.40212445]),
 array([4.65999851, 4.68191026, 4.58078089, 4.5083169 , 4.26415426,
        4.22385202, 4.09557147, 4.07694232, 4.16041681, 4.25560101,
        4.50701023, 4.40106102]),
 array([4.6603987 , 4.67948621, 4.57966338, 4.50957357, 4.25560129,
        4.22784918, 4.09681142, 4.07511169, 4.16244543, 4.25796749,
        4.51066583, 4.40245628])]

In [25]:
real

array([[4.75, 4.63, 4.51, 4.43, 4.3 , 4.17, 4.11, 4.08, 4.13, 4.19, 4.46,
        4.36],
       [4.66, 4.56, 4.49, 4.4 , 4.27, 4.17, 4.13, 4.11, 4.17, 4.23, 4.5 ,
        4.4 ],
       [4.65, 4.54, 4.47, 4.38, 4.23, 4.13, 4.09, 4.07, 4.13, 4.19, 4.45,
        4.35],
       [4.59, 4.53, 4.46, 4.38, 4.23, 4.15, 4.1 , 4.07, 4.12, 4.17, 4.43,
        4.33],
       [4.57, 4.5 , 4.42, 4.34, 4.19, 4.1 , 4.05, 4.03, 4.09, 4.15, 4.42,
        4.34]])

In [278]:
naive

[array([4.76, 4.69, 4.58, 4.42, 4.3 , 4.13, 4.1 , 4.05, 4.1 , 4.18, 4.45,
        4.36]),
 array([4.76, 4.69, 4.58, 4.42, 4.3 , 4.13, 4.1 , 4.05, 4.1 , 4.18, 4.45,
        4.36]),
 array([4.76, 4.69, 4.58, 4.42, 4.3 , 4.13, 4.1 , 4.05, 4.1 , 4.18, 4.45,
        4.36]),
 array([4.76, 4.69, 4.58, 4.42, 4.3 , 4.13, 4.1 , 4.05, 4.1 , 4.18, 4.45,
        4.36]),
 array([4.76, 4.69, 4.58, 4.42, 4.3 , 4.13, 4.1 , 4.05, 4.1 , 4.18, 4.45,
        4.36])]

In [ ]:
for h in range(0, 5):
    if h == 0:
        target_col = ori_col
    else:
        target_col = [f'{element}_+_{h}' for element in ori_col]
    
    feature_col = [element for element in all_cols if 'window' in element]

    df_train_modified = df_train[:(len_train-h)]
    df_test_modified = df_test.iloc[[0]]
    print(len(df_train_modified))

    X_train, y_train = df_train_modified[feature_col], df_train_modified[target_col]
    X_test, y_test = df_test_modified[feature_col], df_test.iloc[h][ori_col]

X_test